In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, Astra kekgun example

This is the class method for running Xopt.

In [ ]:
from xopt import Xopt
import os

In [ ]:
# Make a proper input file.
YAML = """
xopt: {output_path: temp}

algorithm:
  name: random_sampler
  options: {chunk_size: 4, max_samples: 10}

simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options:
    astra_input_file: templates/kekgun/kekgun.in
    generator_input_file: templates/kekgun/dist004.in

# Note that you can call another file in the top level group:
vocs:
    variables:
      sig_x: [0.05, 1]
      lt: [0.005, 0.07]
      maxe(1): [20, 50]
      phi(1): [-30, 30]
      maxb(1): [0, 0.4]
      maxe(2): [0, 32]
      phi(2): [-180, 180]
      maxb(2): [0, 0.3]
      maxe(3): [0, 32]
      maxe(4): [0, 32]
      phi(3): [-45, 45]
      phi(4): [-45, 45]
      phi(6): [-45, 45]
    constants:
      ipart: 1000
      lspch: true
      zstop: 16.54

    objectives:
      end_core_emit_95percent_x: MINIMIZE
      end_sigma_z: MINIMIZE
    constraints:
      end_sigma_z: [LESS_THAN, 0.0015]
      end_core_emit_95percent_x: [LESS_THAN, 9.0e-07]
      end_sigma_energy: [LESS_THAN, 200000.0]
      end_higher_order_energy_spread: [LESS_THAN, 5000.0]
      end_mean_kinetic_energy: [GREATER_THAN, 90000000.0]
      end_n_particle_loss: [LESS_THAN, 1]
    linked_variables: null

"""

In [ ]:
!mkdir temp

In [ ]:
# Create object
X = Xopt(config=YAML)

# Change some things to make it run faster
print(X.vocs)
X.vocs["constants"]["lspch"] = True
X.vocs["constants"]["ipart"] = 10
X.vocs["constants"]["zstop"] = 0.2

# Show config
X

In [ ]:
# Check random inputs
X.random_inputs()

In [ ]:
%%time
# Do a random evaluate to check that everything will run
output = X.random_evaluate()
output

In [ ]:
# These are the algorithm options
X.algorithm["options"]

In [ ]:
# These are the options in the evaluate function
X.simulation["options"]

# Run Sampler using processes or threads

In [ ]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor

# from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [ ]:
# Show config
X.run(executor)

In [ ]:
# Check for errors
X.results["error"]

# Write this configuration

In [ ]:
X.save("test.yaml")

In [ ]:
!cat test.yaml

# Run with MPI

In [ ]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run -vv test.yaml

# Dask 

In [ ]:
from dask.distributed import Client

executor = Client()
# executor = Client(processes=True)
executor

In [ ]:
# Wait a few seconds for the Dask cluster to start
from time import sleep

sleep(5)

In [ ]:
# Run some more
X.run(executor)
executor.close()

In [ ]:
X.results["error"]

# Load into Pandas

In [ ]:
import pandas as pd
from glob import glob
import json

In [ ]:
jfiles = glob("temp/*json")
jfiles

In [ ]:
# Load a file
data = json.load(open(jfiles[0]))

data["inputs"]
data["outputs"]
data["error"]

In [ ]:
inputs = pd.DataFrame(data["inputs"])
inputs

In [ ]:
outputs = pd.DataFrame(data["outputs"])
outputs

# Cleanup

In [ ]:
import shutil

os.chmod("dask-worker-space/", 0o777)
shutil.rmtree("dask-worker-space/")
shutil.rmtree("temp/")
os.chmod("test.yaml", 0o777)
os.remove("test.yaml")